In [1]:
#Hyperparameters:

SHOULD_USE_SEED = True
RADIUS = 10000
BATCH_SIZE = 700
EPSILON = 0.1
SIGMA = 0.0001
MAX_ITERS = 100

TRAIN_FILE = "snli_1.0_train.txt"
VAL_FILE = "snli_1.0_dev.txt"
TEST_FILE = "snli_1.0_test.txt"


In [2]:
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import numpy as np

from data.data_creator import data_create_SNLI

if SHOULD_USE_SEED:
    np.random.seed(42)


## Dataset 

In [3]:
data, labels = data_create_SNLI()

print("Example a premise and hypothesis from the database ", data[TRAIN_FILE][0])
print("Number of instances in the training database: ", len(data[TRAIN_FILE]))

Example a premise and hypothesis from the database  A person on a horse jumps over a broken down airplane., A person is training his horse for a competition.
Number of instances in the training database:  550152


In [4]:
#Splitting the dataset by using the original split

x_train, x_val, x_test = data[TRAIN_FILE], data[VAL_FILE], data[TEST_FILE]
y_train, y_val, y_test = labels[TRAIN_FILE], labels[VAL_FILE], labels[TEST_FILE]

## Black-box model - Multinomial Naive Bayes classifier

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect_text = TfidfVectorizer(use_idf = False)
x_vec_train = vect_text.fit_transform(x_train)

clf = MultinomialNB().fit(x_vec_train, y_train)

In [6]:
preds = clf.predict(vect_text.transform(x_val))

In [7]:
print('Val accuracy', metrics.accuracy_score(y_val, preds))

Val accuracy 0.5363


## Instance to explain

In [8]:
x_explain = x_test[1]#"the movie's thesis -- elegant technology for the masses -- is surprisingly refreshing ."
print('x to explain: ',x_explain)
print('Predicted class: ', clf.predict(vect_text.transform([x_explain]))[0])
print('True class: ', y_test[1])
print('Predict probablilities: ', clf.predict_proba(vect_text.transform([x_explain]))[0])


x to explain:  This church choir sings to the masses as they sing joyous songs from the book at a church., The church is filled with song.
Predicted class:  neutral
True class:  entailment
Predict probablilities:  [5.30589613e-06 3.11126962e-01 1.87523271e-01 5.01344461e-01]


## Building MeLime model:

In [9]:
import nltk
from torch.utils.data import DataLoader
def tokenizer(x):
    return x.split()
dl_train = [tokenizer(x) for x in x_train]

In [10]:
from gen_models.word2vec_gen import Word2VecGen, Word2VecEncoder
#The radius is <radius> most similar words
generator = Word2VecGen(encoder = Word2VecEncoder(dl_train), corpus = x_train, radius = RADIUS, tokenizer = tokenizer)

In [11]:
from interpretable_local_models.statistics_model import StatisticsLocalModel
tokenized_x_explain = x_explain.split()
y_p_explain = max(clf.predict_proba(vect_text.transform([x_explain]))[0])
explainer_model = StatisticsLocalModel(y_p_explain, len(tokenized_x_explain), tokenizer)
print(tokenized_x_explain)

['This', 'church', 'choir', 'sings', 'to', 'the', 'masses', 'as', 'they', 'sing', 'joyous', 'songs', 'from', 'the', 'book', 'at', 'a', 'church.,', 'The', 'church', 'is', 'filled', 'with', 'song.']


In [12]:
from MeLime.model import MeLimeModel

def transform_func(x):
    return vect_text.transform([x])

model = MeLimeModel(black_box_model = clf,gen_model =generator, batch_size = BATCH_SIZE, epsilon_c = EPSILON, 
                    sigma = SIGMA, explainer_model = explainer_model, transform_func = transform_func, 
                    max_iters = MAX_ITERS, tokenizer = tokenizer)

## Explaining the instance

In [13]:
res, sentences_with_probs = model.forward(x_explain)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Plotting results

In [ ]:
ax = StatisticsLocalModel.plot_explaination(res)


In [ ]:
import seaborn as sns
StatisticsLocalModel.plot_sentence_heatmap(res)

## Plotting most favorable and contrary samples phrases:

Favorable sentence - a generated sentence using Word2VecGen that improves the model's confidence in its 
prediction on the original sentence.

Contrary samples - a generated sentence using Word2VecGen that decrease the model's confidence in its prediction on the original sentence and <b>might even change its prediction on the generated sentence</b>.

### Most contrary samples phrases:



In [ ]:
sorted(sentences_with_probs, key = lambda x: x[1])[:5]

### Most favorable samples phrases:

In [ ]:
sorted(sentences_with_probs, key = lambda x: x[1], reverse = True)[:5]